# Import several modules and perform initial setup:

This cell imports various modules necessary for data reduction, visualization, and data handling.

-  If you want to subtract dar, bias, and flat, the only thing you have to do is to include them in the reduction section. Use the cell containing (star1=red.reduce(20, crbox='lacosmic', bias=bias, display=t)), and deactivate the cell containing (imcr=red.crrej(im,crbox='lacosmic',display=t,crsig=10)) and replace imcr with star1 in the coming cells.
  - Note: Subtracting dark reduces the quality of your spectra. I don't if this is because I have a bad dark. To be Examined.
  - This notebook vsky-calibration. However, the wave-cal method and the sky-cal method are still in progress.

In [33]:
#!pip install scikit-image

In [34]:
#!pip install scikit-learn

In [35]:
#!pip install PyQt6
#!pip install PySide6
#!pip install PyQt5
#!pip install PySide2




In [1]:
from pyvista import imred, tv, spectra, stars, slitmask, image
import numpy as np
import pdb
import copy
import matplotlib.pyplot as plt
import os
from astropy.table import vstack
import pandas as pd
import importlib

# Matplotlib configuration for notebook interaction:

Configures how matplotlib displays plots—either within the notebook or in a separate interactive window. The tv.TV() call appears to initialize a tool for displaying images interactively.

In [2]:
# Use these lines if you are running the notebook yourself. Matplotlib
# window will open outside the notebook, which is the desired behavior so
# you can have a single display tool, which you should leave open. Other
# plot windows will also appear outside the notebook, which you can close
# as desired
%matplotlib qt
t=tv.TV()
plotinter=True

# following lines only for fully non-interactive demo of notebook
#%matplotlib inline
#plotinter=False
#t=None

# Setup for image reduction:

Specifies the directory containing images and initializes an image reducer for the KOSMOS instrument.

In [3]:
# put directory name with images here
indir='UT230909'
red=imred.Reducer('KOSMOS',dir=indir)

INSTRUMENT: KOSMOS   config: 
  will use format:  UT230909/*{:04d}.f*.fits*
         gain:  [0.6]    rn: [5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    43    20  4056 
  2105    43    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


this cell show you the table of your file directory.

Displays the log of the image reduction operations in the notebook, limited to the first 10 entries.


In [4]:
red.log().show_in_notebook(display_length=40)

idx,FILE,DATE-OBS,OBJNAME,RA,DEC,EXPTIME
0,Flat_SEG3G2.0001.fits,2023-09-09T01:31:47.843412,,6:56:00.00,75:00:00.00,2.0
1,Flat_SEG3G2.0002.fits,2023-09-09T01:33:24.544267,,6:56:00.00,75:00:00.00,1.0
2,Flat_SEG3G2.0003.fits,2023-09-09T01:34:47.647398,,6:56:00.00,75:00:00.00,0.5
3,Flat_SEG3R2.0004.fits,2023-09-09T01:38:24.858222,,6:56:00.00,75:00:00.00,0.5
4,Flat_EMPTY_TEST.0005.fits,2023-09-09T01:40:21.135840,,6:56:00.00,75:00:00.00,0.5
5,Flat_SEG3R1.0006.fits,2023-09-09T01:43:41.792202,,6:56:00.00,75:00:00.00,0.5
6,Flat_SEG3R1.0008.fits,2023-09-09T01:47:35.037138,,6:56:00.00,75:00:00.00,0.5
7,SEG3G2.0010.fits,2023-09-09T02:19:53.351408,Seg3G2-Mask,21:21:32.11,19:08:59.20,20.0
8,SEG3G2.0011.fits,2023-09-09T02:33:52.513049,Seg3G2-Mask,21:21:34.14,19:09:06.69,20.0
9,SEG3G2.0012.fits,2023-09-09T02:36:34.405725,Seg3G2-Mask,21:21:34.14,19:09:06.69,11.2


Read in a single mask observation, along with calibration data

# Reduce and display various astronomical images:

Performs reduction on different sets of data: a star image, a flat field image, and a lamp image.

In [5]:
im = red.reduce(20) # A simple reduction of your statr.
flat1 = red.reduce(22) # reading the flat 
arcs=red.sum([23,24]) # Reading your lamp

  Reading file: UT230909\SEG3G2.0020.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Reading file: UT230909\Flat_SEG3G2.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Reading file: UT230909\Ne_120s_SEG3G2.0023.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Reading file: UT230909\Ne_360s_SEG3G2.0024.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  combining data with sum....
  calculating uncertainty....


# Display the science image using an interactive tool:

Checks if the interactive display tool t is initialized and if so, clears the current view and displays the science image im.

In [6]:
# Display the science image
if t is not None:
    t.clear()
    t.tv(im)

Display the flat field image

In [7]:
# Display the flat
if t is not None:
    t.clear()
    t.tv(flat1)

Display the lamp image.
Displays the lamp image arcs, using the same method.

In [8]:
# Display the lamp
if t is not None:
    t.clear()
    t.tv(arcs)

# Cosmic ray rejection in the image:

The cell below ejects cosmic rays and does nothing eles.

If crbox is given as a 2-element list, then a box of this shape is run over the image. At each location, the median in the box is determined. For each pixel in the box, if the value is larger than crsig*uncertainty (where uncertainty is taken from the input.uncertainty.array), the pixel is replaced by the median. If crsig is a list, then multiple passes are done with successive values of crsig (which should then be decreasing), but only neighbors of previously flagged CRs are tested. Affected pixels are flagged in input.bitmask 

If crbox=’lacosmic’, the LA Cosmic routine, as implemented in astroscrappy is run on the image, with default options, but objlim, fsmode, and inbkg can be specified.

crsig (list/float, default 5, threshold for CR rejection if using spatial)

In [9]:
imcr=red.crrej(im,crbox='lacosmic',display=t,crsig=10)

  zapping CRs with astroscrappy detect_cosmics
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 


# Use the cells below if you are in need of subtracting flat, bias, or dark.

In [10]:
flatims=[21,22]
flat=red.mkflat(flatims,spec=True) #spec=Trues,display=None,littrow=False, trim= False,)

  Reading file: UT230909\Flat_SEG3G2.0021.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Flat_SEG3G2.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [11]:
if t is not None:
    t.tv(flat)

In [12]:
biastims=[74,75,76,77,78]
bias=red.mkbias(biastims,display=None)

  Reading file: UT230909\Bias.0074.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0075.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0076.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0077.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0078.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [13]:
if t is not None:
    t.tv(bias)

In [14]:
darktims=[94,95,96]
dark=red.mkdark(darktims,display=None)

  Reading file: UT230909\dark.0094.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\dark.0095.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\dark.0096.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [15]:
if t is not None:
    t.tv(dark)

*Question*
- Do you know what this flat error is about?

In [16]:
#t.clear()
star1=red.reduce(20, crbox='lacosmic', bias=bias, flat=flat, dark=dark, display=t)
#t.clear()

  Reading file: UT230909\SEG3G2.0020.fits
  subtracting overscan vector 
  subtracting overscan vector 
  See bias box (solid outlines applied to dashed regions of the same color), and cross section. 
   To continue, hit space in display window (p for debug) 
  subtracting bias...
  subtracting dark...
  zapping CRs with astroscrappy detect_cosmics
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 
  flat fielding...


C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\imred.py:765: RuntimeWarning: divide by zero encountered in divide
  corr.data /= flat.data
C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\imred.py:766: RuntimeWarning: divide by zero encountered in divide
  corr.uncertainty.array /= flat.data
C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\function_base.py:158: RuntimeWarning: invalid value encountered in multiply
  y *= step
C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\function_base.py:168: RuntimeWarning: invalid value encountered in add
  y += start


  See flat-fielded image and original with - (minus) key.
   To continue, hit space in display window (p for debug) 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Find slit edges from flat, and fit polynomials to locations

# Find and display slits on the flat field image:

Initializes a Trace object for tracing spectral lines, possibly transposing the image for better handling. It finds and displays slits on the flat field image flat1 with a threshold and signal-to-noise parameter.

In [17]:
# this does not take mkflat().
trace=spectra.Trace(transpose=True)
t.tvclear()
bottom,top = trace.findslits(flat1,display=t,thresh=0.5,sn=True)

An over view of your trace (the informations it contains). Outputs the internal variables of the trace object, useful for debugging or understanding the current state of the object.

In [18]:
vars(trace)

{'type': 'Polynomial1D',
 'degree': 2,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-50, 50),
 'model': [Polynomial([205.39570053,   6.70214955, -19.58890376], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([353.47561566,   6.31656815, -13.19157469], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([500.31128681,   6.02780702,  -9.3204817 ], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([725.38596237,   6.39430102,  -3.60916434], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([934.65809011,   6.48687832,   1.55820243], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1023.40503861,    6.46322524,    3.67504815], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1089.0796024,    6.4892661,    5.2677399], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1154.51935672,    6.50896277,    6.81061458], d

# Read slit mask from a file and sort:

Load and read your kmsfile. A kmsfile is a file containing the information used to make a mask-slit. 
Thus, it is important to load the right file. Sorting by 'YMM' is very important for the coming steps. 
It arrangement the table data from top to bottom to correspond to the slit arrangement from left to right.
This allows us to match the slit position to its corresponding kmsfile very easily.
Reads slit mask data from a specified file and sorts it, likely preparing it for further processing.

In [19]:
kmsfile='kms/Copy of kosmos.23.seg3g2.kms'
targets = slitmask.read_kms(kmsfile,sort='YMM')

Outputs the table targets which was read from the .kms file, to examine its contents.

Transform your to panda table formate. This will allow you to easily modify the table. 

In [20]:
targets # A look at your table

ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
str7,str2,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64
TARG113,NN,STRAIGHT,4.0,4.0,0.0,212134.279,191220.25,0.683,0.683,-5.252,-34.315
TARG112,NN,STRAIGHT,4.0,4.0,0.0,212140.986,191141.81,0.683,0.683,-21.469,-27.768
TARG114,NN,STRAIGHT,4.0,4.0,0.0,212123.429,191104.21,0.683,0.683,20.979,-21.323
TARG111,NN,STRAIGHT,0.9,10.0,0.0,212132.879,191002.48,0.154,1.707,-1.87,-10.801
TARG110,NN,STRAIGHT,0.9,10.0,0.0,212127.784,190908.67,0.154,1.707,10.451,-1.61
TARG109,NN,STRAIGHT,0.9,10.0,0.0,212128.763,190845.78,0.154,1.707,8.085,2.297
TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
TARG115,NN,STRAIGHT,4.0,4.0,0.0,212132.347,190735.58,0.683,0.683,-0.582,14.272


# Convert the targets table to a pandas DataFrame:

Converts the targets table to a pandas DataFrame and displays it, allowing for easier manipulation and visualization using pandas tools.

In [21]:
df = targets.to_pandas()

In [22]:
df # A look at your table in panda formate

,ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
0,TARG113,NN,STRAIGHT,4.0,4.0,0.0,212134.279,191220.25,0.683,0.683,-5.252,-34.315
1,TARG112,NN,STRAIGHT,4.0,4.0,0.0,212140.986,191141.81,0.683,0.683,-21.469,-27.768
2,TARG114,NN,STRAIGHT,4.0,4.0,0.0,212123.429,191104.21,0.683,0.683,20.979,-21.323
3,TARG111,NN,STRAIGHT,0.9,10.0,0.0,212132.879,191002.48,0.154,1.707,-1.870,-10.801
4,TARG110,NN,STRAIGHT,0.9,10.0,0.0,212127.784,190908.67,0.154,1.707,10.451,-1.610
5,TARG109,NN,STRAIGHT,0.9,10.0,0.0,212128.763,190845.78,0.154,1.707,8.085,2.297
6,TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
7,TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
8,TARG115,NN,STRAIGHT,4.0,4.0,0.0,212132.347,190735.58,0.683,0.683,-0.582,14.272
9,TARG106,NN,STRAIGHT,0.9,10.0,0.0,212133.934,190720.87,0.154,1.707,-4.422,16.780


# Remove specified rows from DataFrame:

If all of your slits don't have a light source or you want only to extract a portion of science images. 
Look at your science image and see what positional index it responds to; is it slit number 1, 2, 3, etc.
This position will be the same as in the table above. Note that in Python, index counting starts at zero (0).
In the cell below you can remove the slits you are not interested in. This will not affect your original kmsfile.

In [23]:
# Specify the indices of the rows you want to remove
rows_to_remove = [0,1,2,3,4,5,8,10,12,14,15]

# Remove the specified rows
df_cleaned = df.drop(rows_to_remove)

Outputs the cleaned DataFrame df_cleaned to verify the rows have been removed correctly.

In [24]:
df_cleaned

,ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
6,TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
7,TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
9,TARG106,NN,STRAIGHT,0.9,10.0,0.0,212133.934,190720.87,0.154,1.707,-4.422,16.780
11,TARG104,NN,STRAIGHT,0.9,10.0,0.0,212136.755,190631.54,0.154,1.707,-11.247,25.194
13,TARG102,NN,STRAIGHT,0.9,10.0,0.0,212134.103,190555.71,0.154,1.707,-4.831,31.313


# Filter and process data based on index values:

Now you only have your slits of interest. Take the index value of each slit and create a list containing their slit width from the trace.

In [25]:
# Give your index value
in_dex = [6, 7, 9, 11, 13]

# Create a new list to store the filtered lines
filtered_rows = []

for index, line in enumerate(trace.rows):
    if index in in_dex:
        filtered_rows.append(line)

# Replace the original trace.rows with the filtered list
trace.rows = filtered_rows


In [26]:
trace.rows

[[1089, 1128], [1154, 1193], [1352, 1391], [1542, 1581], [1681, 1720]]

If you needed to create Trace from scratch, instantiate one with rows=, lags=, sc= (transpose= determines whether input image needs to be transposed to get wavelength changing along columns).

Note the skip= keyword, which can be used when tracing to speed things up by only computing centroids for every skip pixel, taking a median around these pixels of width to skip pixels. The default is skip=10

sc0 is your center reference point. sc0 = 2028 is the center of KOSMOS TESCOPE MIRRO.

Set rows = trace.rows, it specifies the boundaries of the slits you will be tracing.


#  This cell initializes a new Trace object using various parameters:

In [27]:
# Create your own trace from scratch.
trace1=spectra.Trace(lags=range(-150,150),
                    rows= trace.rows ,transpose=red.transpose, rad=5, degree= 3) #[1585,1545],# 1372 #1545-1585
vars(trace1)

{'type': 'Polynomial1D',
 'degree': 3,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-150, 150),
 'rows': [[1089, 1128],
  [1154, 1193],
  [1352, 1391],
  [1542, 1581],
  [1681, 1720]],
 'model': None,
 'sigmodel': None,
 'sc0': None}

# This cell configures and performs a trace operation on an image using predefined rows and parameters:

 - `srow`: Specifies the rows (1110, 1173, 1372, 1564, 1697) where the spectra are located on the image. These rows are manually set and correspond to positions of interest, likely stars or other astronomical objects, observed in the imaging software DS9.
- `trace1.trace()`: Calls the trace method on the `trace1` object. The parameters used are:
  - `imcr`: The image data on which the trace will be applied.
  - `srow`: The list of starting rows for the trace, indicating where each trace should begin.
   - `skip=10`: This parameter likely skips some data points to reduce computational load or noise effects, improving the trace accuracy or performance.
  - `gaussian = True`: Applies a Gaussian fit to the trace data, which helps in accurately capturing the profile of spectral lines.
  - `display=t`: Displays the tracing process, possibly in an interactive viewer or output log, for visualization and verification.
  - `rad= 5`: Sets the radius of the trace to 5, defining the thickness of the area around each row from `srow` to be included in the trace. `vars(trace1)`: Outputs the attributes of `trace1` after the tracing operation, which is useful for debugging and understanding the changes made to the object during the trace.


*Future developments*
- There may be a need for developing a code that automate the "srow" process.

*Question*
- Why reduce refrence stars?

In [28]:
#Trace
srow= [1110, 1173, 1372,1564,1697]   #1173,746,954,1041, # list to allow for multiple spectra on an image, manually set

# rad is setting the width of your trace. It will take the center position to be the star position given taking from DS9.
trace1.trace(star1,srow,skip=10, display=t, rad= 5)
vars(trace1)

C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\function_base.py:158: RuntimeWarning: invalid value encountered in multiply
  y *= step
C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\function_base.py:168: RuntimeWarning: invalid value encountered in add
  y += start


C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\spectra.py:1200: RuntimeWarning: invalid value encountered in divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\spectra.py:1205: RuntimeWarning: invalid value encountered in divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<rad))[0]


  Tracing row: 1697
  See trace. Hit space bar to continue....


{'type': 'Polynomial1D',
 'degree': 3,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': array([    0.        , 27576.87109375,  1537.84667969, ...,
          445.91418457,  1486.26672363,   346.99194336]),
 'rad': 5,
 'transpose': True,
 'lags': range(-150, 150),
 'rows': [[1089, 1128],
  [1154, 1193],
  [1352, 1391],
  [1542, 1581],
  [1681, 1720]],
 'model': [<Polynomial1D(3, c0=1105.86339103, c1=0.00002215, c2=0.0000007, c3=0.)>,
  <Polynomial1D(3, c0=1170.18725639, c1=-0.00124282, c2=0.00000073, c3=0.)>,
  <Polynomial1D(3, c0=1374.23039276, c1=-0.00645038, c2=0.00000218, c3=0.)>,
  <Polynomial1D(3, c0=1571.65889376, c1=-0.01145581, c2=0.00000328, c3=0.)>,
  <Polynomial1D(3, c0=1709.53254929, c1=-0.0154988, c2=0.00000438, c3=0.)>],
 'sigmodel': None,
 'sc0': 2048,
 'spectrum2d': array([[-3.34514868e+03, -4.00964453e+03, -4.45851416e+03, ...,
         -9.68437805e+02, -1.07595679e+03, -9.81393463e+02],
        [ 4.45251611e+03,  4.61554004e+03, -4.89653656e+02, ...,
         -1.69828394e+03

Using the derived traces, extract the slitless for arcs.
This cell performs a 2D extraction from the image of the lamp.

In [29]:
arcec1=trace1.extract2d(arcs, display=t)

extracting: 
 1089-1128
 1154-1193
 1352-1391
 1542-1581
 1681-1720
  See extraction window(s). Hit space bar to continue....


# This cell checks if the number of target objects matches the number of slits and updates FITS headers:

Load the targets file, and add XMM and YMM for each slit to headers of each extracted image. REQUIRES A PERFECT MATCH BETWEEN INPUT TARGETS AND IDENTIFIED SLITS!

 - The condition `if len(targets) == len(bottom):` checks whether the list of targets and the list of bottom positions have the same length.
   This is crucial because each target needs to correspond to a slit for proper data association and header updates.
 - Inside the loop `for arc, target in zip(arcec1, targets):`, it iterates over pairs of extracted arc data (`arc`) and target data (`target`) simultaneously.
   `zip(arcec1, targets)` creates these pairs, ensuring that each arc is associated with its corresponding target.
 - For each pair, `arc.header['XMM'] = target['XMM']` and `arc.header['YMM'] = target['YMM']` updates the FITS header of the arc data with the X and Y coordinates from the target data.
   These headers ('XMM' and 'YMM') represent physical positions or moment measurements that are essential for further analysis or cataloging.
 - If the lengths do not match, it executes the `else:` block where it prints an error message:
  `print('ERROR, number of identified slits does not match number of targets')`.
   This error handling is important to avoid mismatches that could lead to incorrect data processing or analysis.


In [30]:
if len(targets) == len(bottom) : 
    for arc,target in zip(arcec1,targets) :
        arc.header['XMM'] = target['XMM']
        arc.header['YMM'] = target['YMM']
else :
    print('ERROR, number of identified slits does not match number of targets')

Now loop through each extracted arc to do wavelength calibration. This requires a little effort because the change in the location of the slit relative to the default saved wavelength calibration is significant enough that it can be a challenge to automatically find the lines, since the change in spectrum is more than a simple shift (and, in fact, more than a shift + dispersion change).

However, a simple shift is usually enough to identify some of the lines, and these can be used to bootstrap the wavelength solution; the initial identification is easier given an estimate of the shift from the mask design.

You can use identify() to do the iteration. On the first pass, only central lines may be correctly identified. Use 'l' and 'r' to remove lines to the left and right of the identified lines. Then use 'i' to iterate, i.e., allow it to re-identify lines (i just returns True to allow you to iterate). When happy with solution, use ' ' to move onto the final 2D wavelength calibration.

In [66]:
#from astropy.io import ascii, fits
#lamp_template = ascii.read('kosmos-arc/templates/NeRed1.18-ctr.spec')  
#lamp_template['wave']
#plt.close()
#print(lamp_template['flux'])

In [67]:
# Extract the wavelength column
#wavelength = lamp_template['wave']

# Save the wavelength data to a new file
#with open('new_wave_lamps/wavelength_data.dat', 'w') as file:
    #for wave in wavelength:
       # file.write(f'{wave}  NeI \n')


# This cell performs wavelength calibration for each arc spectrum in 'arcec1' using a predefined wavelength solution file:

 - `for i, arc in enumerate(arcec1):` iterates through each arc spectrum in 'arcec1', with `i` as the index and `arc` as the spectrum.
 - `wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')`: Initializes a wavelength calibration object `wav` using a FITS file that likely contains reference wavelength data.
 - `nrow=arc.shape[0]`: Retrieves the number of rows in the `arc` data, which represents the slit width of each slit in the spectral data.
 - `shift=int(arc.header['XMM'])`: Extracts a shift value from the FITS header 'XMM'. This shift is used as an initial guess for aligning the observed spectrum with the reference.
 - `lags=np.arange(shift-400, shift+400)`: Defines a range of lags around the initial shift for more refined wavelength calibration, adjusting the position to better align with reference features.
 - The `while` loop (`while iter:`) continues until no further identification improvements are made:
   - `iter = wav.identify(...)`: Calls the identify method on `wav` to align the spectrum. It adjusts the calibration based on the central row of the spectrum (`arc[nrow//2]`) and updates the `iter` flag based on whether further adjustments are needed.
   - Parameters like `plot=True` and `plotinter=True` enable visualizations of the calibration process to manually inspect and adjust the fit.
   - After the first iteration, `lags` is reset to a standard range (`np.arange(-150, 150)`), which likely focuses on fine-tuning around a new estimated center.
   - `plt.close()`: Closes the plot to clear memory and avoid overlapping plots.
 - After exiting the loop, `wav.identify(...)` is called again on the whole `arc` to perform a 2D wavelength solution across the slit by sampling at 5 locations (determined by `nrow//5`).
 - The final `plt.close()` again ensures that all plots are closed, maintaining clean output without residual figures.


*Questions:*
- Where did the shift values come from?
- nrow is 39 or 40 the max range of slit width. nrow//2 is fiting wave at the center of the slits right?
- Why does the lags range comefrom?
- Why don't we use center line for center, highline for high, and lowline for low? inteated of all in one file.

In [31]:
for i,arc in enumerate(arcec1) :
    
    wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
    nrow=arc.shape[0] # this is referring to the slit width of each slit. 
    # get initial guess at shift from reference using XMM (KOSMOS red low!)
    shift=int(arc.header['XMM'])#*-22.5) # 500 #-wav.pix0)
    lags=np.arange(shift-400,shift+400)

    iter = True
    while iter :
        iter = wav.identify(arc[nrow//2],plot=True,plotinter=True,
                            lags=lags,thresh=5,file='new_wave_lamp/new_neon_red_center.dat', rad = 5) # new_neon_red_center.dat, I created this file from KOSMO'S lamp plot.
        lags=np.arange(-300,300) #'new_wave_lamps/new_neon_red_center.dat'
        plt.close()
        
    # Do the 2D wavelength solution, sampling 5 locations across slitlet
    wav.identify(arc,plot=True,nskip=nrow//5,thresh=5)
    plt.close()

  rms:    0.177 Angstroms (50 lines)
  cross correlating with reference spectrum using lags between:  -405 394
  Derived pixel shift from input wcal:  [-255.75494749]


C:\Users\abdul\OneDrive\Desktop\Summer_24\pyvista\python\pyvista\spectra.py:695: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  See identified lines.
  rms:   75.740 Angstroms (27 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:   75.740 Anstroms
  input from plot window...
  rms:    2.172 Anstroms
  input from plot window...
  rms:    1.315 Anstroms
  input from plot window...
  rms:    0.157 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -300 299
  Derived pixel shift from input wcal:  [1.09838538e-09]
  See identified lines.
  rms:    0.133 Angstroms (27 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish a

Now extract the science frame.

This cell performs a two-dimensional spectral extraction from an image:
 - `t.clear()`: Clears any existing data or settings in the object `t`, which might be a graphical or plotting interface used to display outputs. This ensures a fresh start for new operations.
 - `out1 = trace1.extract2d(imcr, display=t)`: Executes a 2D extraction method on the image `imcr` using the `trace1` object. 
   - `imcr`: The image from which the spectra are being extracted. This could be a pre-processed image ready for spectral analysis.
   - `display=t`: This parameter likely enables the visualization of the extraction process. If `t` is linked to a display or graphical interface, this would allow real-time observation of the extraction, which is useful for ensuring it is proceeding as expected or for debugging purposes.
The output of this extraction is stored in the variable `out1`, which will contain the extracted spectral data. This data can then be used for further analysis or processing.


In [32]:
t.clear()
out1=trace1.extract2d(star1,display=t)

extracting: 
 1089-1128
 1154-1193
 1352-1391
 1542-1581
 1681-1720
  See extraction window(s). Hit space bar to continue....


# This cell visualizes and processes spectral data after extraction, involving both plotting and adding wavelength information:

 - `t.clear()`: Clears any previous data or settings in the visualization tool `t`, ensuring a fresh start for new operations.
 - `plt.figure()`: Opens a new figure window for plotting, setting up a visualization space for the upcoming plots.
 - The loop `for i, (o, a) in enumerate(zip(out1, arcec1)):` iterates over paired spectral data from `out1` and `arcec1`:
   - `out1` and `arcec1` are iterated concurrently using `zip`, pairing each spectrum `o` from `out1` with its corresponding calibration data `a` from `arcec1`.
   - `print(o.shape)`: Prints the shape of the spectrum `o`, which gives an insight into the dimensions of the spectral data (e.g., number of spectral lines, width of the spectrum).
   - `print(a.wave)`: Prints the wavelength data from the object `a`, which is essential for understanding the spectral range covered.
   - `o.add_wave(a.wave)`: Adds the wavelength information from `a` to the spectral data `o`, enabling calibrated spectral analysis.
   - `name = o.header["FILE"].split(".")[0]`: Extracts a base file name from the 'FILE' header of the spectrum `o`, typically used for output naming.
   - `print(name)`: Outputs the extracted name for verification or tracking purposes.
   - `plt.plot(o.wave[10], o.data[10])`: Plots a specific line (the 10th line) of the spectrum `o`, visualizing the intensity data against its wavelength. This can help visually assess the quality or characteristics of the spectrum.
   - The commented out lines `#t.tv(o)` and `#t.tv(o.wave)` suggest that there was an option to visualize the spectrum directly using the `t` tool, but this is not currently active.
   - The commented out line `#o.write(name + "_{:d}.fits".format(i))` indicates that saving the processed data to FITS files was considered but is not currently executed, perhaps for debugging or development purposes.

In [40]:
t.clear()
plt.figure()
for i,(o,a) in enumerate(zip(out1,arcec1)) :
    print(o.shape)
    print(a.wave)
    o.add_wave(a.wave)
    name = o.header["FILE"].split(".")[0]
    print(name)
    #t.tv(o)
    #t.tv(o.wave)
    plt.plot(o.wave[10],o.data[10])
    #o.write(name + "centroid_{:03d}.fits".format(i))

(39, 4096)
[[9348.05639497 9347.03345181 9346.01044509 ... 5264.909361
  5264.09267559 5263.27615458]
 [9348.13049702 9347.10741773 9346.08427505 ... 5264.84923865
  5264.03238752 5263.2157006 ]
 [9348.2016192  9347.17840967 9346.1551369  ... 5264.79215517
  5263.97514553 5263.15829993]
 ...
 [9348.84675094 9347.82261655 9346.79842009 ... 5264.65944208
  5263.84133521 5263.02339114]
 [9348.81357768 9347.78951914 9346.76539842 ... 5264.7087189
  5263.890706   5263.072856  ]
 [9348.77742456 9347.75344774 9346.72940865 ... 5264.7610346
  5263.94312288 5263.12537417]]
SEG3G2
appending uncertainty
appending bitmask
appending wave
(39, 4096)
[[9721.13760795 9720.10907843 9719.08048752 ... 5601.28642378
  5600.44954828 5599.61282786]
 [9721.09060852 9720.06213649 9719.03360308 ... 5601.24916716
  5600.41220666 5599.57540119]
 [9721.04563094 9720.01721443 9718.98873652 ... 5601.2143065
  5600.37726512 5599.54037871]
 ...
 [9720.71939566 9719.69168857 9718.66391973 ... 5601.45464245
  5600.6172

As desired, move on to 1D extraction

This cell refines spectral extraction by reloading the spectra module, finding peaks, and extracting specific regions:
 - `importlib.reload(spectra)`: Reloads the `spectra` module to ensure that the latest changes in the module are active during this execution, particularly useful during development when adjustments are frequently made.
 - `def model(x): return x*0.`: Initially defines a simple model function that returns zero for any input, effectively setting a baseline or placeholder behavior for further calculations.
 - `fig=plt.figure()`: Creates a new figure for plotting, which will be used to plot spectral data.
 - The loop `for i in range(len(out1)):` iterates over all extracted spectra in `out1`.
   - `trace1 = spectra.Trace(transpose=False)`: Initializes a new `Trace` object for each spectrum in `out1` with transposition turned off.
   - `trace1.rows = [0, out1[i].data.shape[0]]`: Sets the row indices for the trace operation, spanning the entire vertical extent of the spectrum.
   - `trace1.index = [0]`: Initializes an index for the trace, usually specifying the starting point or reference for the trace operation.
   - `peak, ind = trace1.findpeak(out1[i], thresh=10, sort=True)`: Attempts to find peaks in the spectral data which exceed a threshold value; sorting may prioritize or order the peaks.
   - Conditional processing if peaks are found:
     - If peaks are identified (`if len(peak) > 0:`), a new model function is defined where it returns a constant value equal to the highest peak.
     - `trace1.model = [model]`: Assigns this model to the trace object for use in further extraction.
     - `spec=trace1.extract(out1[i], rad=5, back=[[-10,-5],[10,5]], display=t)`: Extracts the spectrum using the new model, specifying radial bounds and background subtraction regions. The `display=t` might be used for visual feedback.
     - `plt.figure(fig)`: Ensures that subsequent plots are made on the originally created figure.
     - `spec.wave = out1[i].wave[peak]`: Assigns wavelength data to `spec.wave` based on the peak locations, aligning extracted spectral data with its corresponding wavelengths.
     - `print(spec.wave[0].shape, spec.data[0].shape)`: Prints the shape of the wave and data arrays for debugging or verification.
     - `plt.plot(spec.wave[0], spec.data[0])`: Plots the extracted spectrum.
   - If no peaks are found, it prints a message indicating no peak was found for the current slit index.
   - The commented out `plt.draw()` suggests that immediate plotting or dynamic updates were considered but are currently disabled.


*Question*
- trace1.model = [model] is the trace1 model to the peaks right?

In [34]:
trace.rows

[[1089, 1128], [1154, 1193], [1352, 1391], [1542, 1581], [1681, 1720]]

In [35]:
l=[1089, 1154, 1352, 1542, 1681]

In [36]:
nrow=out1[0].shape[0]//2
nrow

19

In [39]:
importlib.reload(spectra)
def model(x) :
    return x*0.
l=[1089, 1154, 1352, 1542, 1681]
fig=plt.figure()
for i in range(len(out1)) :
    #fig = plt.figure() 
    nrow=out1[i].shape[0]//2
    center = l[i]+nrow
    trace1 = spectra.Trace(transpose=False, sc0=center)
    trace1.rows = [0,out1[i].data.shape[0]]
    trace1.index = [0]
    peak,ind = trace1.findpeak(out1[i],thresh=5,sort=True)
    if len(peak) > 0:
        def model(x) :
            return x*0. + peak[0]
        trace1.model = [model]
        spec=trace1.extract(out1[i],rad=5,back=[[-10,-5],[10,5]], display=t) # This line is important because you are extracting sky here.
        plt.figure(fig)
        spec.wave = out1[i].wave[peak]
        print(spec.wave[0].shape,spec.data[0].shape)
        plt.plot(spec.wave[0],spec.data[0])
        #spec.write("spec_centroid_without_sky" + "_{:03d}.fits".format(i))
    else :
        print('no peak found for slit: ',i)
    #plt.draw()

looking for peaks using 200 pixels around 1108, threshhold of 5.000000
peaks:  [20]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
appending uncertainty
appending bitmask
appending wave
looking for peaks using 200 pixels around 1173, threshhold of 5.000000
peaks:  [18]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
appending uncertainty
appending bitmask
appending wave
looking for peaks using 200 pixels around 1371, threshhold of 5.000000
peaks:  [19]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
appending uncertainty
appending bitmask
appending wave
looking for peaks using 200 pixels around 1561, threshhold of 5.000000
peaks:  [21, 36, 7]
aperture/fiber:  [1, 2, 0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
appending uncertainty
appendin

In [38]:
importlib.reload(spectra)
def model(x) :
    return x*0.
l=[1089, 1154, 1352, 1542, 1681]
fig=plt.figure()
for i in range(len(out1)) :
    #fig = plt.figure() 
    nrow=out1[i].shape[0]//2
    center = l[i]+nrow
    trace1 = spectra.Trace(transpose=False, sc0= center)
    trace1.rows = [0,out1[i].data.shape[0]]
    trace1.index = [0]
    peak,ind = trace1.findpeak(out1[i],thresh=10,sort=True)
    if len(peak) > 0:
        def model(x) :
            return x*0. + peak[0]
        trace1.model = [model]
        spec=trace1.extract(out1[i],rad=5, display=t) # This line is important because you are extracting sky here.
        plt.figure(fig)
        spec.wave = out1[i].wave[peak]
        print(spec.wave[0].shape,spec.data[0].shape)
        plt.plot(spec.wave[0],spec.data[0])
        #spec.write("spec_centroid_with_sky" + "_{:03d}.fits".format(i))
    else :
        print('no peak found for slit: ',i)
    #plt.draw()

looking for peaks using 200 pixels around 1108, threshhold of 10.000000
peaks:  [20]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
appending uncertainty
appending bitmask
appending wave
looking for peaks using 200 pixels around 1173, threshhold of 10.000000
peaks:  [18]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
appending uncertainty
appending bitmask
appending wave
looking for peaks using 200 pixels around 1371, threshhold of 10.000000
peaks:  [19]
aperture/fiber:  [0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
appending uncertainty
appending bitmask
appending wave
looking for peaks using 200 pixels around 1561, threshhold of 10.000000
peaks:  [21, 36, 7]
aperture/fiber:  [1, 2, 0]
  extracting ... 
  See extraction window(s). Hit space bar to continue....

(4096,) (4096,)
appending uncertainty
appe

Adjusting the calibration using skylines.

This cell performs spectral peak identification and extraction with model adjustments, and reloads necessary modules:
 - `import copy`: Imports the `copy` module to allow deep copying of objects, which is used to duplicate complex data structures without reference issues.
 - `importlib.reload(spectra)`: Reloads the `spectra` module, ensuring any updates to the module's code are applied immediately. This is particularly useful during development or iterative testing sessions.
 - `def model(x): return x*0.`: Defines a baseline model function that outputs zero regardless of input, serving as a default or placeholder model.
 - `fig = plt.figure()`: Creates a new figure for plotting. This will be used to plot spectral data if needed.
 - The loop `for i in range(len(out1)):` iterates over all spectra in `out1`.
   - `trace3 = spectra.Trace(transpose=False)`: Initializes a new `Trace` object without transposing the data matrix, tailored for spectral analysis.
   - `trace3.rows = [0, out1[i].data.shape[0]]`: Specifies the row indices for tracing, covering the entire spectrum's vertical range.
   - `trace3.index = [0]`: Sets the starting index for the trace operations, possibly indicating the initial point for spectral processing.
   - `peak, ind = trace3.findpeak(out1[i], thresh=10, sort=True)`: Searches for peaks in the spectral data that surpass a threshold intensity, sorting them likely by intensity or significance.
   - Conditional processing if peaks are identified:
     - If peaks are found (`if len(peak) > 0:`), a custom model function is redefined to return a constant value equal to the highest peak.
     - `trace3.model = [model]`: Assigns this new model to the `trace3` object for subsequent extraction operations.
     - `spec = trace3.extract(out1[i], rad=4, display=None)`: Performs spectral extraction using the defined model, with a specified radius and without display; adjustments might focus on minimizing background interference.
     - `plt.figure(fig)`: Ensures that subsequent plots are made on the originally created figure.
     - `spec.wave = out1[i].wave[peak]`: Aligns the extracted spectral data with wavelength information based on detected peaks.
     - `swav = copy.deepcopy(wav)`: Creates a deep copy of a wavelength calibration object `wav`, allowing modifications without affecting the original.
     - `swav.skyline(spec, thresh=0.5, linear=False, inter=plotinter, file='pyvista/data/sky/skyline.dat')`: Applies a skyline calibration to the spectrum, using parameters to fine-tune the process such as threshold and interpolation.
     - Print statements (`print(wav.model)` and `print(swav.model)`) output the model details before and after modifications to track changes.
 - If no peaks are found, it prints a message indicating the absence of detectable peaks for the current spectrum.
 - The commented-out lines like `#plt.plot(spec.wave[0], spec.data[0])` and `#plt.draw()` suggest optional plotting or visual updates that are currently disabled for perhaps clarity or performance reasons during debugging.


*Question*
- What are the criteria sued for collecting their skylines or skylines in general? I am using my own skylines.

In [48]:
import copy
importlib.reload(spectra)
def model(x) :
    return x*0.
l=[1089, 1154, 1352, 1542, 1681]
fig=plt.figure()
for i in range(len(out1)) :
    nrow=out1[i].shape[0]//2
    center = l[i]+nrow
    trace3 = spectra.Trace(transpose=False, sc0=center)
    trace3.rows = [0,out1[i].data.shape[0]]
    trace3.index = [0]
    peak,ind = trace3.findpeak(out1[i],thresh=10,sort=True)
    if len(peak) > 0:
        def model(x) :
            return x*0. + peak[0]
        trace3.model = [model]
        spec=trace3.extract(out1[i],rad=4, display=None) #,display=t) # back=[[-10,-5],[5,10]
        plt.figure(fig)
        spec.wave = out1[i].wave[peak]
        swav=copy.deepcopy(wav)
        swav.skyline( spec, thresh=0.5 , linear= False, inter=plotinter, file= 'new_wave_lamp/skyline.dat') #'new_wave_lamp/OHll.dat'), # skyline.dat, this file needs further examination.
        #print(wav.model) # 'pyvista/data/sky/OHll.dat'
        
        #print(swav.model)
        
        #print(spec.model)
        #print(spec.wave[0].shape,spec.data[0].shape)
        #plt.plot(spec.wave[0],spec.data[0])
    else :
        print('no peak found for slit: ',i)
    #plt.draw()

looking for peaks using 200 pixels around 1108, threshhold of 10.000000
peaks:  [20]
aperture/fiber:  [0]
  extracting ... 

  See identified lines.
  rms:    1.542
rejecting 1 points from 20 total: 
  rms:    1.116
rejecting 1 points from 20 total: 
  See 2D wavecal fit. Enter space in plot window to continue

looking for peaks using 200 pixels around 1173, threshhold of 10.000000
peaks:  [18]
aperture/fiber:  [0]
  extracting ... 

  See identified lines.
  rms:    1.066
rejecting 1 points from 19 total: 
  rms:    0.301
rejecting 1 points from 19 total: 
  See 2D wavecal fit. Enter space in plot window to continue

looking for peaks using 200 pixels around 1371, threshhold of 10.000000
peaks:  [19]
aperture/fiber:  [0]
  extracting ... 

  See identified lines.
  rms:    2.086
rejecting 1 points from 19 total: 
  rms:    1.360
rejecting 1 points from 19 total: 
  See 2D wavecal fit. Enter space in plot window to continue

looking for peaks using 200 pixels around 1561, threshhold of

In [42]:
vars(swav)

{'type': 'chebyshev2D',
 'degree': 3,
 'ydegree': 2,
 'waves': array([6300.31 , 6363.78 , 6498.74 , 6533.05 , 6544.03 , 6553.626,
        6562.76 , 6863.97 , 6889.3  , 6900.81 , 6912.64 , 6923.19 ,
        6939.54 , 7003.87 , 7316.289, 7571.75 , 7794.12 , 8399.175]),
 'waves_order': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 'orders': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'index': array([0], dtype='>i8'),
 'pix0': array([122.28475135]),
 'pix': array([3332.08020488, 3267.31799649, 3118.34038159, 3080.68345078,
        3075.81571946, 3058.35860601, 3048.62666481, 2757.83949389,
        2729.04490153, 2715.79609089, 2708.57702227, 2697.85237222,
        2677.50119563, 2621.74371564, 2340.65789922, 2108.03056118,
        1891.07884543, 1308.55647452]),
 'y': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [95]:
print(wav.model) # 'pyvista/data/sky/OHll.dat'

print(swav.model)

Model: Chebyshev2D
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
X_Degree: 3
Y_Degree: 2
X_Domain: (0, 4096)
Y_Domain: (0, 1)
Parameters:
           c0_0              c1_0                c2_0        ...           c2_2                    c3_2         
    ----------------- ------------------ ------------------- ... ----------------------- -----------------------
    7465.909159961359 -1850.700400838105 -30.201422989891793 ... -0.00025181083822133646 -0.00033508712038022296
Model: Chebyshev2D
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
X_Degree: 3
Y_Degree: 2
X_Domain: (0, 4096)
Y_Domain: (0, 1)
Parameters:
           c0_0               c1_0               c2_0        ...         c2_2                  c3_2         
    ------------------ ------------------ ------------------ ... ------------------- -----------------------
    2492.1442732624523 -1850.700400838105 -10.37792693459779 ... -10.377926934598978 -0.00033508712038022296


In [43]:
wav.model

<Chebyshev2D(3, 2, c0_0=7465.90924339, c1_0=-1850.70039393, c2_0=-30.20146791, c3_0=130.84696216, c0_1=0.0830531, c1_1=0.02504797, c2_1=0.04479919, c3_1=0.06917512, c0_2=-0.00041669, c1_2=-0.00001272, c2_2=-0.00025181, c3_2=-0.00033509, x_domain=(0, 4096), y_domain=(0, 1))>

In [44]:
swav.model

<Chebyshev2D(3, 2, c0_0=2492.14430111, c1_0=-1850.70039393, c2_0=-10.3779496, c3_0=66.00292722, c0_1=-2492.14430111, c1_1=2.68802315, c2_1=0.04479919, c3_1=-66.00292722, c0_2=2492.14430111, c1_2=-2.68802315, c2_2=-10.3779496, c3_2=-0.00033509, x_domain=(0, 4096), y_domain=(0, 1))>

In [45]:
from astropy.io import ascii, fits

In [61]:
filename = 'spec_with_sky_00.fits'

In [62]:
    hdul = fits.open(filename)
    #print(hdul[0].header)
    hdul.info()

Filename: spec_with_sky_00.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      68   ()      
  1                1 ImageHDU         7   (4096, 1)   float64   
  2  UNCERT        1 ImageHDU         9   (4096, 1)   float64   
  3  BITMASK       1 ImageHDU        10   (4096, 1)   int32 (rescales to uint32)   
  4  WAVE          1 ImageHDU         8   (4096, 2)   float64   


In [63]:
print(hdul[0].header)

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                    8 / array data type                                NAXIS   =                    0 / number of array dimensions                     EXTEND  =                    T                                                  OBSERVAT= 'APO'                / Per the IRAF observatory list.                 TELESCOP= '3.5m'                                                                INSTRUME= 'kosmos'             / Instrument name                                LATITUDE= +3.2780361000000E+01 / Latitude of telescope base                     LONGITUD= -1.0582041700000E+02 / Longitude of telescope base                    UTC-TAI = -37.0                / UTC = TAI + UTC_TAI(seconds)                   UT1-TAI = -36.99               / UT1 = TAI + UT1_TAI(seconds)                   LST     = '20:02:28.388'       / Local Apparent Sidereal time                   OBJNAME = 'Seg3G2-Mask'        / Object 

In [64]:
import astropy.io.fits as fits
import matplotlib.pyplot as plt

# Open the FITS file
filename = 'spec_with_sky_00.fits'
hdul = fits.open(filename)

# Extract the wavelength and flux data
flux = hdul[1].data.flatten()    # Assuming the flux data is in the second HDU
wavelength = hdul[4].data[0]  # Assuming the wavelength data is in the fifth HDU

# Plot the spectrum
plt.figure(figsize=(10, 6))
plt.plot(wavelength, flux, label='Spectrum')
plt.xlabel('Wavelength')
plt.ylabel('Flux')
plt.title('Spectrum from FITS file')
plt.legend()
plt.grid(True)
plt.show()

# Close the FITS file
hdul.close()


In [97]:
import astropy.io.fits as fits
import matplotlib.pyplot as plt

# Open the FITS file
filename = 'spec_02.fits'
hdul = fits.open(filename)

# Extract the flux data from the second HDU
flux = hdul[1].data.flatten()  # Assuming the flux data is in the second HDU

# Extract the wavelength data from the fifth HDU
# Assuming the wavelength data is in the first column and has the same length as the flux data
wavelength = hdul[4].data[0]

# Ensure the wavelength and flux arrays have the same shape
if wavelength.shape[0] != flux.shape[0]:
    raise ValueError(f"The wavelength and flux arrays do not have the same length: {wavelength.shape[0]} vs {flux.shape[0]}")

# Plot the spectrum
plt.figure(figsize=(10, 6))
plt.plot(wavelength, flux, label='Spectrum')
plt.xlabel('Wavelength')
plt.ylabel('Flux')
plt.title('Spectrum from FITS file')
plt.xscale('log')  # Set x-axis to logarithmic scale
plt.yscale('log')  # Set y-axis to logarithmic scale
plt.legend()
plt.grid(True)
plt.show()

# Close the FITS file
hdul.close()


In [51]:
f1 = 'spec_1.fits'

In [66]:
file = 'spec_with_sky_00.fits'
huld = fits.open(file)
huld.info()

Filename: spec_with_sky_00.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      68   ()      
  1                1 ImageHDU         7   (4096, 1)   float64   
  2  UNCERT        1 ImageHDU         9   (4096, 1)   float64   
  3  BITMASK       1 ImageHDU        10   (4096, 1)   int32 (rescales to uint32)   
  4  WAVE          1 ImageHDU         8   (4096, 2)   float64   


In [67]:
huld[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
OBSERVAT= 'APO'                / Per the IRAF observatory list.                 
TELESCOP= '3.5m'                                                                
INSTRUME= 'kosmos'             / Instrument name                                
LATITUDE= +3.2780361000000E+01 / Latitude of telescope base                     
LONGITUD= -1.0582041700000E+02 / Longitude of telescope base                    
UTC-TAI = -37.0                / UTC = TAI + UTC_TAI(seconds)                   
UT1-TAI = -36.99               / UT1 = TAI + UT1_TAI(seconds)                   
LST     = '20:02:28.388'       / Local Apparent Sidereal time                   
OBJNAME = 'Seg3G2-Mask'     

In [71]:
plt.figure()
plt.plot(huld[4].data[0],huld[1].data[0])


In [87]:
filename = 'adjusted_spec_with_sky.fits'
hdul = fits.open(filename)

# Extract the wavelength and flux data
flux = hdul[1].data.flatten()    # Assuming the flux data is in the second HDU
wavelength = hdul[4].data[0]  # Assuming the wavelength data is in the fifth HDU

# Plot the spectrum
plt.figure(figsize=(10, 6))
plt.plot(wavelength, flux, label='Spectrum')
plt.xlabel('Wavelength')
plt.ylabel('Flux')
plt.title('Spectrum from FITS file')
plt.legend()
plt.grid(True)
plt.show()

In [104]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

# Load the FITS and skyline data
fits_file_path = 'spec_with_sky_00.fits'
skyline_file_path = 'new_wave_lamp/telluric_lines_test.dat'

with fits.open(fits_file_path) as hdul:
    spectra_data = hdul[1].data[0]
    wavelength_data = hdul[4].data[0]

skyline_wavelengths = np.loadtxt(skyline_file_path)

# Extract the first spectrum and wavelength for demonstration
spectrum = spectra_data.copy()  # Make a copy to keep the original intact
wavelength = wavelength_data.copy()  # Make a copy to keep the original intact
original_wavelength = wavelength_data.copy()  # Keep the original for comparison

# Plotting
fig, ax1 = plt.subplots()

ax1.plot(original_wavelength, spectrum, label='Original Spectrum')
for wl in skyline_wavelengths:
    ax1.axvline(wl, color='red', linestyle='--', alpha=0.5)
ax1.set_ylabel('Intensity')
ax1.set_title('Spectra and Skyline Correspondence')
ax1.legend()

# Variables to store clicked points
selected_troughs = []
assigned_skylines = []

# Click event handler
def on_click(event):
    x_click = event.xdata
    
    if event.key == 'l' and x_click is not None:
        # Identify the closest trough in the spectrum
        closest_trough_idx = (np.abs(wavelength - x_click)).argmin()
        closest_trough = wavelength[closest_trough_idx]
        
        selected_troughs.append(closest_trough)
        ax1.axvline(closest_trough, color='blue', linestyle='-', alpha=0.7)
        fig.canvas.draw()
        
    elif event.key == 'r' and x_click is not None:
        if selected_troughs:
            # Assign to nearest skyline
            closest_skyline = skyline_wavelengths[np.abs(skyline_wavelengths - x_click).argmin()]
            assigned_skylines.append(closest_skyline)
            
            # Adjust the spectrum trough to this skyline wavelength
            last_trough = selected_troughs.pop()
            trough_idx = np.where(wavelength == last_trough)[0][0]
            wavelength[trough_idx] = closest_skyline
            
            # Plot adjusted trough
            ax1.axvline(closest_skyline, color='green', linestyle='-', alpha=0.7)
            fig.canvas.draw()
            
            output_fits_file_path = 'adjusted_spectrum.fits'
            
            hdu_spectrum = fits.PrimaryHDU(spectrum)
            hdu_wavelength = fits.ImageHDU(wavelength, name='WAVELENGTH')
            
            hdul = fits.HDUList([hdu_spectrum, hdu_wavelength])
            hdul.writeto(output_fits_file_path,)   # overwrite=True     
# Connect the key press event to the handler
fig.canvas.mpl_connect('key_press_event', on_click)

# Show the plot
plt.show()

# Save the adjusted spectrum and wavelength in a FITS file


# Plot original and adjusted spectra for comparison
fig, ax2 = plt.subplots()
ax2.plot(original_wavelength, spectrum, label='Original Spectrum', alpha=0.7)
ax2.plot(wavelength, spectrum, label='Adjusted Spectrum', alpha=0.7)
ax2.set_ylabel('Intensity')
ax2.set_title('Original vs Adjusted Spectrum')
ax2.legend()

plt.show()

In [41]:
from astropy.io import fits
f = fits.open('adjusted_spectrum.fits')
f1 = fits.open('spec_with_sky_00.fits')
f.info()

Filename: adjusted_spectrum.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   (4096,)   float64   
  1  WAVELENGTH    1 ImageHDU         7   (4096,)   float64   


In [42]:
plt.figure()
plt.plot(f[1].data, f[0].data)
plt.plot(f1[4].data[0], f1[1].data[0])

In [3]:
#%%writefile 'new_wave_lamp/telluric_lines.dat'
9869.0 O2
6940.7 H2O
6956.4 H2O
6889.0 H2O
7605.0 O2
7901.8 H2O
7908.8 H2O
7928.6 H2O
7962.2 H2O
8133.8 H2O
8162.5 H2O
8178.2 H2O
8198.9 H2O
8228.8 H2O
8243.5 H2O
8256.5 H2O
8288.5 H2O
8294.6 H2O
8304.7 H2O
8321.4 H2O
9019.2 H2O
9082.9 H2O
9121.2 H2O
9139.1 H2O
9176.8 H2O
9318.8 H2O

Writing new_wave_lamp/telluric_lines.dat
